In [13]:
# -*- coding: utf-8 -*-
import os
import sys
#import urllib2
import requests
import re
from lxml import etree


def StringListSave(save_path, filename, slist):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    path = save_path+"/"+filename+".txt"
    with open(path, "w+") as fp:
        for s in slist:
            fp.write("%s\t\t%s\n" % (s[0].encode("utf8"), s[1].encode("utf8")))

def Page_Info(myPage):
    '''Regex'''
    mypage_Info = re.findall(r'<div class="titleBar" id=".*?"><h2>(.*?)</h2><div class="more"><a href="(.*?)">.*?</a></div></div>', myPage, re.S)
    return mypage_Info

def New_Page_Info(new_page):
    '''Regex(slowly) or Xpath(fast)'''
    # new_page_Info = re.findall(r'<td class=".*?">.*?<a href="(.*?)\.html".*?>(.*?)</a></td>', new_page, re.S)
    # # new_page_Info = re.findall(r'<td class=".*?">.*?<a href="(.*?)">(.*?)</a></td>', new_page, re.S) # bugs
    # results = []
    # for url, item in new_page_Info:
    #     results.append((item, url+".html"))
    # return results
    dom = etree.HTML(new_page)
    new_items = dom.xpath('//tr/td/a/text()')
    new_urls = dom.xpath('//tr/td/a/@href')
    assert(len(new_items) == len(new_urls))
    return zip(new_items, new_urls)

def Spider(url):
    i = 0
    print("downloading ", url) 
    myPage = requests.get(url).content.decode("gbk")
    # myPage = urllib2.urlopen(url).read().decode("gbk")
    myPageResults = Page_Info(myPage)
    save_path = u"网易新闻抓取"
    filename = str(i)+"_"+u"新闻排行榜"
    StringListSave(save_path, filename, myPageResults)
    i += 1
    for item, url in myPageResults:
        print("downloading ", url) 
        new_page = requests.get(url).content.decode("gbk")
        # new_page = urllib2.urlopen(url).read().decode("gbk")
        newPageResults = New_Page_Info(new_page)
        filename = str(i)+"_"+item
        StringListSave(save_path, filename, newPageResults)
        i += 1


if __name__ == '__main__':
    print ("start")
    start_url = "http://news.163.com/rank/"
    Spider(start_url)
    print ("end")

start
downloading  http://news.163.com/rank/
downloading  http://news.163.com/special/0001386F/rank_whole.html
downloading  http://news.163.com/special/0001386F/rank_news.html
downloading  http://news.163.com/special/0001386F/rank_ent.html
downloading  http://news.163.com/special/0001386F/rank_sports.html
downloading  http://money.163.com/special/002526BH/rank.html
downloading  http://news.163.com/special/0001386F/rank_tech.html
downloading  http://news.163.com/special/0001386F/rank_auto.html
downloading  http://news.163.com/special/0001386F/rank_lady.html
downloading  http://news.163.com/special/0001386F/rank_house.html
downloading  http://news.163.com/special/0001386F/game_rank.html
downloading  http://news.163.com/special/0001386F/rank_travel.html
downloading  http://news.163.com/special/0001386F/rank_edu.html
end


In [2]:
#!/usr/bin/env python
# encoding=utf-8
import requests
import re
import codecs
from bs4 import BeautifulSoup
from openpyxl import Workbook
wb = Workbook()
dest_filename = '电影.xlsx'
ws1 = wb.active
ws1.title = "电影top250"

DOWNLOAD_URL = 'http://movie.douban.com/top250/'


def download_page(url):
    """获取url地址页面内容"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
    }
    data = requests.get(url, headers=headers).content
    return data


def get_li(doc):
    soup = BeautifulSoup(doc, 'html.parser')
    ol = soup.find('ol', class_='grid_view')
    name = []  # 名字
    star_con = []  # 评价人数
    score = []  # 评分
    info_list = []  # 短评
    for i in ol.find_all('li'):
        detail = i.find('div', attrs={'class': 'hd'})
        movie_name = detail.find(
            'span', attrs={'class': 'title'}).get_text()  # 电影名字
        level_star = i.find(
            'span', attrs={'class': 'rating_num'}).get_text()  # 评分
        star = i.find('div', attrs={'class': 'star'})
        star_num = star.find(text=re.compile('评价'))  # 评价

        info = i.find('span', attrs={'class': 'inq'})  # 短评
        if info:  # 判断是否有短评
            info_list.append(info.get_text())
        else:
            info_list.append('无')
        score.append(level_star)

        name.append(movie_name)
        star_con.append(star_num)
    page = soup.find('span', attrs={'class': 'next'}).find('a')  # 获取下一页
    if page:
        return name, star_con, score, info_list, DOWNLOAD_URL + page['href']
    return name, star_con, score, info_list, None


def main():
    url = DOWNLOAD_URL
    name = []
    star_con = []
    score = []
    info = []
    while url:
        doc = download_page(url)
        movie, star, level_num, info_list, url = get_li(doc)
        name = name + movie
        star_con = star_con + star
        score = score + level_num
        info = info + info_list
    for (i, m, o, p) in zip(name, star_con, score, info):
        col_A = 'A%s' % (name.index(i) + 1)
        col_B = 'B%s' % (name.index(i) + 1)
        col_C = 'C%s' % (name.index(i) + 1)
        col_D = 'D%s' % (name.index(i) + 1)
        ws1[col_A] = i
        ws1[col_B] = m
        ws1[col_C] = o
        ws1[col_D] = p
    wb.save(filename=dest_filename)


if __name__ == '__main__':
    main()

In [4]:
import requests
import os
import time
import threading
from bs4 import BeautifulSoup


def download_page(url):
    '''
    用于下载页面
    '''
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0"}
    r = requests.get(url, headers=headers)
    r.encoding = 'gb2312'
    return r.text


def get_pic_list(html):
    '''
    获取每个页面的套图列表,之后循环调用get_pic函数获取图片
    '''
    soup = BeautifulSoup(html, 'html.parser')
    pic_list = soup.find_all('li', class_='wp-item')
    for i in pic_list:
        a_tag = i.find('h3', class_='tit').find('a')
        link = a_tag.get('href')
        text = a_tag.get_text()
        get_pic(link, text)


def get_pic(link, text):
    '''
    获取当前页面的图片,并保存
    '''
    html = download_page(link)  # 下载界面
    soup = BeautifulSoup(html, 'html.parser')
    pic_list = soup.find('div', id="picture").find_all('img')  # 找到界面所有图片
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0"}
    create_dir('pic/{}'.format(text))
    for i in pic_list:
        pic_link = i.get('src')  # 拿到图片的具体 url
        r = requests.get(pic_link, headers=headers)  # 下载图片，之后保存到文件
        with open('pic/{}/{}'.format(text, pic_link.split('/')[-1]), 'wb') as f:
            f.write(r.content)
            time.sleep(1)   # 休息一下，不要给网站太大压力，避免被封


def create_dir(name):
    if not os.path.exists(name):
        os.makedirs(name)


def execute(url):
    page_html = download_page(url)
    get_pic_list(page_html)


def main():
    create_dir('pic')
    queue = [i for i in range(1, 72)]   # 构造 url 链接 页码。
    threads = []
    while len(queue) > 0:
        for thread in threads:
            if not thread.is_alive():
                threads.remove(thread)
        while len(threads) < 5 and len(queue) > 0:   # 最大线程数设置为 5
            cur_page = queue.pop(0)
            url = 'http://meizitu.com/a/more_{}.html'.format(cur_page)
            thread = threading.Thread(target=execute, args=(url,))
            thread.setDaemon(True)
            thread.start()
            print('{}正在下载{}页'.format(threading.current_thread().name, cur_page))
            threads.append(thread)


if __name__ == '__main__':
    main()

MainThread正在下载1页
MainThread正在下载2页
MainThread正在下载3页
MainThread正在下载4页
MainThread正在下载5页


Exception in thread Thread-6:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
Exception in thread Thread-7:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
Exception in thread Thread-8:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
Exception in thread Exception in thread Thread-10:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
Thread-9:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
        httplib_response = self._make_request(httplib_response = self._make_request(
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request

  File "D:\anaconda\lib\site-packages\urllib3\connectionpool

MainThread正在下载6页
MainThread正在下载7页
MainThread正在下载8页
MainThread正在下载9页
MainThread正在下载10页


Exception in thread Thread-12Exception in thread Exception in thread Thread-14:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
:
Thread-11Exception in thread Thread-13:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
Traceback (most recent call last):
:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
      File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
httplib_response = self._make_request(
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
Exception in thread Thread-15:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
        httplib_response = self._make_request(
  File "D:\anaconda\lib\site-packages\urllib3\connec

MainThread正在下载11页
MainThread正在下载12页
MainThread正在下载13页
MainThread正在下载14页


...

            r = adapter.send(request, **kwargs)six.raise_from(e, None)
version, status, reason = self._read_status()
Exception in thread   File "D:\anaconda\lib\site-packages\requests\adapters.py", line 498, in send

  File "<string>", line 3, in raise_from
Thread-74      File "D:\anaconda\lib\http\client.py", line 268, in _read_status
:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\requests\adapters.py", line 439, in send
        raise ConnectionError(err, request=request)httplib_response = self._make_request(    
resp = conn.urlopen(
requests.exceptions    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
.  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line

MainThread正在下载70页


  File "<ipython-input-4-6dbb2dd08638>", line 13, in download_page
six.raise_from(e, None)
return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。  File "<string>", line 3, in raise_from

    
During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\requests\adapters.py", line 439, in send
resp = conn.urlopen(
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 726, in urlopen
    version, status, reason = self._read_status()
  File "D:\anaconda\lib\http\client.py", line 268, in _read_status
    retries = retries.increment(
  File "D:\anaconda\lib\site-packages\urllib3\util\retry.py", line 410, in increment
    resp = conn.urlopen(
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 726, in urlopen
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "D:\anaconda\lib\socket.py", line 669, in readinto
    retries = retr

MainThread正在下载71页


        httplib_response = conn.getresponse()
  File "D:\anaconda\lib\site-packages\requests\api.py", line 76, in get
resp = conn.urlopen(
  File "D:\anaconda\lib\site-packages\urllib3\connectionpool.py", line 726, in urlopen
  File "D:\anaconda\lib\http\client.py", line 1347, in getresponse
            return request('get', url, params=params, **kwargs)
  File "D:\anaconda\lib\site-packages\requests\api.py", line 61, in request
retries = retries.increment(
  File "D:\anaconda\lib\site-packages\urllib3\util\retry.py", line 410, in increment
    response.begin()
  File "D:\anaconda\lib\http\client.py", line 307, in begin
httplib_response = conn.getresponse()
  File "D:\anaconda\lib\http\client.py", line 1347, in getresponse
        return session.request(method=method, url=url, **kwargs)
  File "D:\anaconda\lib\site-packages\requests\sessions.py", line 530, in request
raise six.reraise(type(error), error, _stacktrace)
  File "D:\anaconda\lib\site-packages\urllib3\packages\six.py", line 

In [7]:
import requests
from bs4 import BeautifulSoup


class GetBossData(object):
    """爬取10页的Boss直聘职位数据"""
    domain = 'https://www.zhipin.com'
    base_url = 'https://www.zhipin.com/c101280600/?query='
    position = ''
    # 代理IP地址
    proxies_ip = '58.220.95.30'
    proxies_port = '10174'

    def __init__(self, position):
        self.position = position

    def get_url_html(self, url, cookie):
        """请求页面html"""
        ip_url = self.proxies_ip + ':' + str(self.proxies_port)
        proxies = {'http': 'http://' + ip_url, 'https': 'https://' + ip_url}
        header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
            'cookie': cookie
        }
        request = requests.get(url=url, headers=header, proxies=proxies, timeout=3)
        html = False
        if request.status_code == 200:
            html = request.content
        return html

    def run(self):
        """执行入口"""
        page_list = range(1, 11)
        # 打开文件，准备写入
        dict_file = open('job.md', 'a', encoding='UTF-8')
        # 清空文件内容
        dict_file.seek(0)
        dict_file.truncate()
        dict_file.write('| 岗位 | 区域 | 薪资 | 年限信息 | 公司名称 | 公司信息 | 链接 |')
        dict_file.write('\n| --- | --- | --- | --- | --- | --- | --- |')
        # 分页爬取数据
        for page in page_list:
            print('开始爬取第' + str(page) + '页数据')
            boss_url = self.base_url + str(self.position) + '&page=' + str(page) + '&ka=page-' + str(page)
            # F12打开调试模式，手动刷新网页获取cookie，然后替换
            if page < 4:
                cookie_val = 'lastCity=101280600; __zp_seo_uuid__=d59649f5-bc8a-4263-b4e1-d5fb1526ebbe; __c=1592469667; __g=-; Hm_lvt_194df3105ad7148dcf2b98a91b5e727a=1592469673; __l=l=%2Fwww.zhipin.com%2Fshenzhen%2F&r=https%3A%2F%2Fwww.google.com%2F&friend_source=0&friend_source=0; toUrl=https%3A%2F%2Fwww.zhipin.com%2F%2Fjob_detail%2F3f35305467e161991nJ429i4GA%7E%7E.html; __a=43955211.1592469667..1592469667.39.1.39.39; Hm_lpvt_194df3105ad7148dcf2b98a91b5e727a=1592530438; __zp_stoken__=7f3aaPCVBFktLe0xkP21%2BJSFCLWILSwx7NEw4bVJkRx8pdBE3JGNmWjVwdx5PXC8rHmN%2BJB0hX1UvTz5VPyMmOhIVHBglVzoxJQIdLQtKR3ZFBFIeazwOByVndHwXBAN%2FXFo7W2BffFxtXSU%3D; __zp_sseed__=Ykg0aQ3ow1dZqyi9KmeVnWrqZXcZ32a4psiagwqme3M=; __zp_sname__=93bf4835; __zp_sts__=1592530479301'
            elif page < 7:
                cookie_val = 'lastCity=101280600; __zp_seo_uuid__=d59649f5-bc8a-4263-b4e1-d5fb1526ebbe; __c=1592469667; __g=-; Hm_lvt_194df3105ad7148dcf2b98a91b5e727a=1592469673; __l=l=%2Fwww.zhipin.com%2Fshenzhen%2F&r=https%3A%2F%2Fwww.google.com%2F&friend_source=0&friend_source=0; toUrl=https%3A%2F%2Fwww.zhipin.com%2F%2Fjob_detail%2F3f35305467e161991nJ429i4GA%7E%7E.html; __a=43955211.1592469667..1592469667.39.1.39.39; Hm_lpvt_194df3105ad7148dcf2b98a91b5e727a=1592530438; __zp_stoken__=7f3aaPCVBFktLe0xkP21%2BJSFCLWILSwx7NEw4bVJkRx8pdBE3JGNmWjVwdx5PXC8rHmN%2BJB0hX1UvTz5VPyMmOhIVHBglVzoxJQIdLQtKR3ZFBFIeazwOByVndHwXBAN%2FXFo7W2BffFxtXSU%3D; __zp_sseed__=Ykg0aQ3ow1dZqyi9KmeVnWrqZXcZ32a4psiagwqme3M=; __zp_sname__=93bf4835; __zp_sts__=1592530514188'
            elif page < 10:
                cookie_val = 'lastCity=101280600; __zp_seo_uuid__=d59649f5-bc8a-4263-b4e1-d5fb1526ebbe; __c=1592469667; __g=-; Hm_lvt_194df3105ad7148dcf2b98a91b5e727a=1592469673; __l=l=%2Fwww.zhipin.com%2Fshenzhen%2F&r=https%3A%2F%2Fwww.google.com%2F&friend_source=0&friend_source=0; toUrl=https%3A%2F%2Fwww.zhipin.com%2F%2Fjob_detail%2F3f35305467e161991nJ429i4GA%7E%7E.html; __a=43955211.1592469667..1592469667.40.1.40.40; Hm_lpvt_194df3105ad7148dcf2b98a91b5e727a=1592530479; __zp_stoken__=7f3aaPCVBFktLCT4uVVV%2BJSFCLWIVPWZyNUk4bVJkR25XXHVeZWNmWjVwd286Sm83HmN%2BJB0hX1UvBiBVRyt9IWQOcRtWSk83fAsfJAtKR3ZFBE5efUl%2FByVndHwXRQN%2FXFo7W2BffFxtXSU%3D; __zp_sseed__=Ykg0aQ3ow1dZqyi9KmeVnd/9vyiSRHrJFoMai+azsb8=; __zp_sname__=93bf4835; __zp_sts__=1592530496863'
            else:
                cookie_val = 'lastCity=101280600; __zp_seo_uuid__=d59649f5-bc8a-4263-b4e1-d5fb1526ebbe; __c=1592469667; __g=-; Hm_lvt_194df3105ad7148dcf2b98a91b5e727a=1592469673; __l=l=%2Fwww.zhipin.com%2Fshenzhen%2F&r=https%3A%2F%2Fwww.google.com%2F&friend_source=0&friend_source=0; toUrl=https%3A%2F%2Fwww.zhipin.com%2F%2Fjob_detail%2F3f35305467e161991nJ429i4GA%7E%7E.html; __a=43955211.1592469667..1592469667.41.1.41.41; __zp_stoken__=7f3aaPCVBFktLc1t4VTp%2BJSFCLWJscnlxSgw4bVJkRw9tLB4pb2NmWjVwdwwgc2l7HmN%2BJB0hX1UvGFZVTH0OdhQQfwxfOyoieW8cOgtKR3ZFBAJYRFMcByVndHwXTwN%2FXFo7W2BffFxtXSU%3D; Hm_lpvt_194df3105ad7148dcf2b98a91b5e727a=1592530497; __zp_sseed__=Ykg0aQ3ow1dZqyi9KmeVnSZKsrhFUU/CYntJcRoFki4=; __zp_sname__=93bf4835; __zp_sts__=1592530514188'
            html = self.get_url_html(boss_url, cookie_val)
            soup = BeautifulSoup(html, 'html.parser')
            # 招聘职位列表
            job_list = soup.select('.job-list ul li')
            for job_li in job_list:
                # 单条职位信息
                url = self.domain + job_li.select('.job-title a')[0].attrs['href']
                title = job_li.select('.job-title a')[0].get_text()
                area = job_li.select('.job-title .job-area')[0].get_text()
                salary = job_li.select('.job-limit .red')[0].get_text()
                year = job_li.select('.job-limit p')[0].get_text()
                company = job_li.select('.info-company h3')[0].get_text()
                industry = job_li.select('.info-company p')[0].get_text()
                info = {
                    'title': title,
                    'area': area,
                    'salary': salary,
                    'year': year,
                    'company': company,
                    'industry': industry,
                    'url': url
                }
                print(info)
                # 写入职位信息
                info_demo = '\n| %s | %s | %s | %s | %s | %s | %s |'
                dict_file.write(info_demo % (title, area, salary, year, company, industry, url))
        dict_file.close()


# 程序主入口
if __name__ == '__main__':
    # 实例化
    job_name = input('请输入职位关键字：').strip()
    if job_name == '':
        print('关键字为空，请重新尝试')
        exit(0)
    gl = GetBossData(job_name)
    # 执行脚本
    gl.run()

请输入职位关键字： 经理


开始爬取第1页数据
开始爬取第2页数据
开始爬取第3页数据
开始爬取第4页数据
开始爬取第5页数据
开始爬取第6页数据
开始爬取第7页数据
开始爬取第8页数据
开始爬取第9页数据
开始爬取第10页数据


In [14]:
import requests
import json
from urllib import parse
import os


class GetLolSkin(object):
    """抓取LOL英雄皮肤"""

    def __init__(self):
        """初始化变量"""
        self.hero_url = 'https://lol.qq.com/biz/hero/champion.js'
        self.hero_detail_url = 'http://lol.qq.com/biz/hero/'
        self.skin_folder = 'skin'
        self.skin_url = 'https://ossweb-img.qq.com/images/lol/web201310/skin/big'

    @staticmethod
    def get_html(url):
        """下载html"""
        request = requests.get(url)
        request.encoding = 'gbk'
        if request.status_code == 200:
            return request.text
        else:
            return "{}"

    def get_hero_list(self):
        """获取英雄的完整信息列表"""
        hero_js = self.get_html(self.hero_url)
        # 删除左右的多余信息，得到json数据
        out_left = "if(!LOLherojs)var LOLherojs={};LOLherojs.champion="
        out_right = ';'
        hero_list = hero_js.replace(out_left, '').rstrip(out_right)
        return json.loads(hero_list)

    def get_hero_info(self, hero_id):
        """获取英雄的详细信息"""
        # 获取js详情
        detail_url = parse.urljoin(self.hero_detail_url, hero_id + '.js')
        detail_js = self.get_html(detail_url)
        # 删除左右的多余信息，得到json数据
        out_left = "if(!herojs)var herojs={champion:{}};herojs['champion'][%s]=" % hero_id
        out_right = ';'
        hero_info = detail_js.replace(out_left, '').rstrip(out_right)
        return json.loads(hero_info)

    def download_skin_list(self, skin_list, hero_name):
        """下载皮肤列表"""
        # 循环下载皮肤
        for skin_info in skin_list:
            # 拼接图片名字
            if skin_info['name'] == 'default':
                skin_name = '默认皮肤'
            else:
                if ' ' in skin_info['name']:
                    name_info = skin_info['name'].split(' ')
                    skin_name = name_info[0]
                else:
                    skin_name = skin_info['name']
            hero_skin_name = hero_name + '-' + skin_name + '.jpg'
            self.download_skin(skin_info['id'], hero_skin_name)

    def download_skin(self, skin_id, skin_name):
        """下载皮肤图片"""
        # 下载图片
        img_url = self.skin_url + skin_id + '.jpg'
        request = requests.get(img_url)
        if request.status_code == 200:
            print('downloading……%s' % skin_name)
            img_path = os.path.join(self.skin_folder, skin_name)
            with open(img_path, 'wb') as img:
                img.write(request.content)
        else:
            print('img error!')

    def make_folder(self):
        """初始化，创建图片文件夹"""
        if not os.path.exists(self.skin_folder):
            os.mkdir(self.skin_folder)

    def run(self):
        # 获取英雄列表信息
        hero_json = self.get_hero_list()
        hero_keys = hero_json['keys']
        # 循环遍历英雄
        for hero_id, hero_code in hero_keys.items():
            hero_name = hero_json['data'][hero_code]['name']
            hero_info = self.get_hero_info(hero_id)
            if hero_info:
                skin_list = hero_info['result'][hero_id]['skins']
                # 下载皮肤
                self.download_skin_list(skin_list, hero_name)
            else:
                print('英雄【%s】的皮肤获取有问题……' % hero_name)


# 程序执行入口
if __name__ == '__main__':
    lol = GetLolSkin()
    # 创建图片存储文件
    lol.make_folder()
    # 执行脚本
    lol.run()

downloading……暗裔剑魔-默认皮肤.jpg
downloading……暗裔剑魔-仲裁圣骑.jpg
downloading……九尾妖狐-默认皮肤.jpg
downloading……九尾妖狐-高丽风情.jpg
downloading……九尾妖狐-暗影妖狐.jpg
downloading……九尾妖狐-焰尾妖狐.jpg
downloading……离群之刺-默认皮肤.jpg
downloading……离群之刺-蜂刺.jpg
downloading……离群之刺-绯红.jpg
downloading……离群之刺-足球宝贝.jpg
downloading……离群之刺-实习护士.jpg
downloading……离群之刺-鬼武姬.jpg
downloading……离群之刺-银色之牙.jpg
downloading……牛头酋长-默认皮肤.jpg
downloading……牛头酋长-暗黑灵魂.jpg
downloading……牛头酋长-金牛座.jpg
downloading……牛头酋长-斗牛士.jpg
downloading……牛头酋长-荒野镖客.jpg
downloading……牛头酋长-巨角幻兽.jpg
downloading……牛头酋长-地狱火.jpg
downloading……殇之木乃伊-默认皮肤.jpg
downloading……殇之木乃伊-法老.jpg
downloading……殇之木乃伊-狂欢.jpg
downloading……殇之木乃伊-情绪摇滚.jpg
downloading……殇之木乃伊-阿木木:'主人不要我了'.jpg
downloading……殇之木乃伊-王子不是我.jpg
downloading……殇之木乃伊-小小骑士.jpg
downloading……殇之木乃伊-殇之机器人.jpg
downloading……冰晶凤凰-默认皮肤.jpg
downloading……冰晶凤凰-双重冰晶.jpg
downloading……冰晶凤凰-联盟之喙.jpg
downloading……冰晶凤凰-重甲猎鹰.jpg
downloading……冰晶凤凰-钢铁之翼.jpg
downloading……冰晶凤凰-黯晶凤凰.jpg
英雄【黑暗之女】的皮肤获取有问题……
英雄【残月之肃】的皮肤获取有问题……
downloading……寒冰射手-默认皮肤.jpg
downloading

KeyboardInterrupt: 

In [15]:
import requests
import bs4
import os


def write_file(file_url, file_type):
    """写入文件"""
    res = requests.get(file_url)
    res.raise_for_status()
    # 文件类型分文件夹写入
    if file_type == 1:
        file_folder = 'nhdz\\jpg'
    elif file_type == 2:
        file_folder = 'nhdz\\mp4'
    else:
        file_folder = 'nhdz\\other'
    folder = os.path.exists(file_folder)
    # 文件夹不存在，则创建文件夹
    if not folder:
        os.makedirs(file_folder)
    # 打开文件资源，并写入
    file_name = os.path.basename(file_url)
    str_index = file_name.find('?')
    if str_index > 0:
        file_name = file_name[:str_index]
    file_path = os.path.join(file_folder, file_name)
    print('正在写入资源文件：', file_path)
    image_file = open(file_path, 'wb')
    for chunk in res.iter_content(100000):
        image_file.write(chunk)
    image_file.close()
    print('写入完成！')


def download_file(web_url):
    """获取资源的url"""
    # 下载网页
    print('正在下载网页： %s...' % web_url)
    result = requests.get(web_url)
    soup = bs4.BeautifulSoup(result.text, "html.parser")
    # 查找图片资源
    img_list = soup.select('.vpic_wrap img')
    if img_list == []:
        print('未发现图片资源！')
    else:
        # 找到资源，开始写入
        for img_info in img_list:
            file_url = img_info.get('bpic')
            write_file(file_url, 1)
    # 查找视频资源
    video_list = soup.select('.threadlist_video a')
    if video_list == []:
        print('未发现视频资源！')
    else:
        # 找到资源，开始写入
        for video_info in video_list:
            file_url = video_info.get('data-video')
            write_file(file_url, 2)
    print('下载资源结束：', web_url)
    next_link = soup.select('#frs_list_pager .next')
    if next_link == []:
        print('下载资料结束！')
    else:
        url = next_link[0].get('href')
        download_file('https:' + url)


# 主程序入口
if __name__ == '__main__':
    web_url = 'https://tieba.baidu.com/f?ie=utf-8&kw=段友之家'
    download_file(web_url)

正在下载网页： https://tieba.baidu.com/f?ie=utf-8&kw=段友之家...
正在写入资源文件： nhdz\jpg\2e2eb9389b504fc20c6c1047f2dde71190ef6d45.jpg
写入完成！
正在写入资源文件： nhdz\jpg\d1160924ab18972b0d6bc58ff1cd7b899e510a26.jpg
写入完成！
正在写入资源文件： nhdz\jpg\728da9773912b31b7057ca6a9118367adab4e1be.jpg
写入完成！
正在写入资源文件： nhdz\jpg\dbb44aed2e738bd447d88176b68b87d6267ff9f8.jpg
写入完成！
正在写入资源文件： nhdz\jpg\3c6d55fbb2fb4316ba6d063f37a4462309f7d301.jpg
写入完成！
正在写入资源文件： nhdz\jpg\8435e5dde71190effc474e99d91b9d16fdfa603c.jpg
写入完成！
正在写入资源文件： nhdz\jpg\0ff41bd5ad6eddc4618011be2edbb6fd5266330b.jpg
写入完成！
正在写入资源文件： nhdz\jpg\d788d43f8794a4c2b998246d19f41bd5ad6e3939.jpg
写入完成！
正在写入资源文件： nhdz\jpg\024f78f0f736afc392b7fc23a419ebc4b7451239.jpg
写入完成！
正在写入资源文件： nhdz\jpg\c75c10385343fbf2ff2b6c65a77eca8065388f75.jpg
写入完成！
正在写入资源文件： nhdz\jpg\7dd98d1001e93901470547786cec54e736d19675.jpg
写入完成！
正在写入资源文件： nhdz\jpg\f703738da977391200fbf943f4198618367ae237.jpg
写入完成！


KeyboardInterrupt: 